In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import os
from dotenv import load_dotenv

# Load env variables
load_dotenv()

# Get DB credentials
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Set up Postgres connection
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Show all rows in output
pd.set_option('display.max_rows', None)

### Business Question:
How has Reddit discussion around Snap’s hiring, layoffs, and AI roles evolved over time?

Tracking post trends by topic over multiple years can reveal public perception shifts around job security, hiring experience, and technical focus areas.

In [2]:
query_1 = '''
SELECT
    year,
    topic_tag,
    COUNT(*) AS post_count
FROM reddit_posts
GROUP BY year, topic_tag
ORDER BY year, post_count DESC;
'''

df_q1 = pd.read_sql(query_1, engine)
df_q1

,year,topic_tag,post_count
0,2012,interview_process,1
1,2013,culture_fit,1
2,2014,ml_ai_roles,4
3,2014,interview_process,4
4,2014,other,1
5,2015,ml_ai_roles,4
6,2015,interview_process,4
7,2015,layoffs_rto,1
8,2015,culture_fit,1
9,2016,interview_process,15


**Insight:**  
Mentions of interviews related to Snap have increased sharply over time, rising from **80 posts in 2021 to 197 in 2024** — a **146% increase**. Layoff-related posts also spiked from **5 in 2021 to 50 in 2024**, signaling growing anxiety about job security. AI/ML role mentions peaked at **92 in 2022** but remained strong with **77 posts in 2024**, showing sustained interest even amid slight decline.

**Recommendation:**  
Snap’s hiring and comms teams should proactively address candidate concerns about layoffs and interviews. A public “Hiring FAQs” resource or more recruiter transparency on social platforms could reduce friction during hiring. Reinforcing AI team stability in messaging may also help retain interest.

**Prediction:**  
If these Reddit trends continue, Snap’s reputation in hiring will increasingly influence offer acceptance. In the competitive AI/ML talent market, even the perception of instability may push top candidates toward firms with clearer, more reassuring narratives.

----------------------------------------------------------------------------------------------------

### Business Question:
Which Snap-related posts on Reddit received the highest engagement (upvotes and comments) across each major theme, and which subreddits did they appear in?

In [7]:
sql_query = '''
WITH topic_summary AS (
    SELECT
        id,
        topic_tag,
        score,
        num_comments,
        created_utc,
        CASE
            WHEN created_utc >= '2023-01-01' THEN 'recent'
            ELSE 'older'
        END AS era
    FROM reddit_posts
    WHERE topic_tag IS NOT NULL
),
exploded AS (
    SELECT 
        id,
        UNNEST(STRING_TO_ARRAY(topic_tag, ',')) AS tag,
        score,
        num_comments,
        era
    FROM topic_summary
),
topic_aggregates AS (
    SELECT
        tag,
        era,
        COUNT(*) AS post_count,
        AVG(score) AS avg_score,
        AVG(num_comments) AS avg_comments
    FROM exploded
    GROUP BY tag, era
),
ranked_topics AS (
    SELECT *,
           RANK() OVER (PARTITION BY era ORDER BY avg_score DESC) AS score_rank
    FROM topic_aggregates
)

SELECT 
    t1.tag,
    t1.era,
    t1.post_count,
    ROUND(t1.avg_score, 1) AS avg_score,
    ROUND(t1.avg_comments, 1) AS avg_comments,
    t2.total_posts_in_era,
    t1.score_rank
FROM ranked_topics t1
JOIN (
    SELECT era, COUNT(*) AS total_posts_in_era
    FROM exploded
    GROUP BY era
) t2 ON t1.era = t2.era
ORDER BY t1.tag, t1.era;
'''

df_q3 = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)
df_q3

,tag,era,post_count,avg_score,avg_comments,total_posts_in_era,score_rank
0,AI roles,older,223,391.7,90.1,705,3
1,AI roles,recent,277,766.2,135.8,951,2
2,culture,older,5,84.4,46.8,705,5
3,culture,recent,19,339.3,132.8,951,5
4,interview,older,364,451.1,90.8,705,2
5,interview,recent,449,649.2,114.4,951,3
6,layoff,older,45,785.2,138.2,705,1
7,layoff,recent,118,879.6,148.2,951,1
8,other,older,68,151.3,104.4,705,4
9,other,recent,88,429.8,149.8,951,4


**Insight:**  
Reddit posts about Snap tagged as “layoffs_rto” nearly **tripled from 45 (older) to 118 posts (recent)**. Their average score jumped from **785 to 880**, and comments increased from **138 to 148** — showing surging concern about job security.  
Likewise, posts about “ml_ai_roles” rose from **223 to 277** with average scores nearly **doubling (392 → 766)** and comment counts increasing to **136 per post**. “interview_process” remained highly active, growing to **449 posts** with an average score of **649**.

These engagement spikes reflect **rising public interest** in Snap’s AI hiring and growing unease around its layoff trends.

**Recommendation:**  
Snap should prioritize messaging strategies and recruiter-facing FAQs addressing its AI roadmap and layoff concerns. High-comment posts indicate confusion or fear — opportunities for transparency and trust-building.

**Prediction:**  
As Snap continues AI expansion, engagement around hiring and layoffs will likely intensify. Without a coordinated communication strategy, Snap risks **allowing viral posts to shape its employer brand**, especially among tech talent evaluating job offers.
